In [ ]:
!pip -q install bokeh==3.5.0 pandas==2.2.2
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared



In [ ]:
code = r"""
# main.py — Bokeh Server: subir CSV y graficar interactivamente
import base64, io
import pandas as pd
from bokeh.io import curdoc
from bokeh.models import (
    ColumnDataSource, Select, Slider, Div, HoverTool,
    CrosshairTool, TapTool, BoxSelectTool, FileInput
)
from bokeh.layouts import column
from bokeh.plotting import figure

def is_numeric(series: pd.Series) -> bool:
    return pd.api.types.is_numeric_dtype(series)

def make_bar_from_categorical(series: pd.Series):
    counts = series.astype("string").value_counts(dropna=False).sort_index()
    cats   = [str(c) for c in counts.index.tolist()]
    vals   = counts.values.tolist()
    return ColumnDataSource(data=dict(x=cats, y=vals)), "Conteo por categoría"

def make_bar_from_numeric(series: pd.Series, bins: int = 10):
    clean = series.dropna()
    if clean.empty:
        return ColumnDataSource(data=dict(x=["Sin datos"], y=[0])), "Sin datos numéricos"
    hist = pd.cut(clean, bins=bins, include_lowest=True)
    counts = hist.value_counts().sort_index()
    edges = counts.index.categories
    labels = [f"{edges[i].left:.3g}–{edges[i].right:.3g}" for i in range(len(edges))]
    return ColumnDataSource(data=dict(x=labels, y=counts.values.tolist())), f"Histograma ({bins} bins)"

def read_csv_bytes(b64value: str) -> pd.DataFrame:
    raw = base64.b64decode(b64value)
    try:
        return pd.read_csv(io.StringIO(raw.decode("utf-8")))
    except Exception:
        return pd.read_csv(io.StringIO(raw.decode("latin-1")))

title_div = Div(text="<h2>Sube un CSV y grafica una columna (Bokeh Server)</h2>", width=900)
hint_div  = Div(text="Elige una columna; si es numérica, ajusta los bins.", width=900)
error_div = Div(text="", width=900, styles={"color": "crimson"})

file_input = FileInput(accept=".csv", multiple=False)
select     = Select(title="Columna", options=[], disabled=True)
slider     = Slider(title="Bins (solo numéricas)", start=5, end=40, step=1, value=10, disabled=True)

source = ColumnDataSource(data=dict(x=[], y=[]))

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,tap,box_select"
p = figure(height=500, width=900, tools=TOOLS, active_scroll="wheel_zoom",
           toolbar_location="above", sizing_mode="stretch_width", x_range=[])
bars = p.vbar(x="x", top="y", width=0.9, source=source, name="bars")

# Estilos de selección
bars.selection_glyph = bars.glyph.clone()
bars.selection_glyph.fill_alpha = 1.0
bars.selection_glyph.line_color = "#000000"
bars.selection_glyph.line_width = 1.5
bars.nonselection_glyph = bars.glyph.clone()
bars.nonselection_glyph.fill_alpha = 0.25

hover = HoverTool(tooltips=[("x", "@x"), ("y", "@y")], renderers=[bars], mode="mouse")
p.add_tools(hover, CrosshairTool())
p.toolbar.active_inspect = hover
p.toolbar.active_tap = p.select_one(TapTool)
p.xaxis.major_label_orientation = 0.9

info_div = Div(text="(Aún no hay datos). Sube un CSV.", width=900)

state = {"df": None}

def compute_and_update():
    df = state["df"]
    col = select.value
    if df is None or not col:
        return
    s = df[col]
    if is_numeric(s):
        slider.disabled = False
        new_source, ylabel = make_bar_from_numeric(s, bins=slider.value)
    else:
        slider.disabled = True
        new_source, ylabel = make_bar_from_categorical(s)

    p.yaxis.axis_label = ylabel
    p.xaxis.axis_label = col
    p.x_range.factors = list(new_source.data["x"])   # ← forzar list
    source.data = dict(new_source.data)              # ← copiar como dict()
    source.selected.indices = []
    info_div.text = "Selecciona una barra (Tap) o usa Box Select. Hover para ver valores."

def on_file_change(attr, old, new):
    error_div.text = ""
    if not file_input.value:
        return
    try:
        df = read_csv_bytes(file_input.value)
        if df.empty or df.columns.empty:
            raise ValueError("CSV vacío o sin columnas.")
        state["df"] = df
        cols = df.columns.astype(str).tolist()
        select.options = cols
        select.value = cols[0]
        select.disabled = False
        slider.disabled = not is_numeric(df[select.value])
        compute_and_update()
    except Exception as e:
        state["df"] = None
        select.options = []
        select.value = ""
        select.disabled = True
        slider.disabled = True
        source.data = dict(x=[], y=[])
        p.x_range.factors = []
        p.yaxis.axis_label = ""
        p.xaxis.axis_label = ""
        info_div.text = "(Aún no hay datos)."
        error_div.text = f"<b>Error al leer CSV:</b> {e}"

def on_select_change(attr, old, new):
    df = state["df"]
    if df is None or not new:
        return
    slider.disabled = not is_numeric(df[new])
    compute_and_update()

def on_slider_change(attr, old, new):
    compute_and_update()

def on_selection_change(attr, old, new):
    inds = source.selected.indices
    if inds:
        i = inds[0]
        x = source.data["x"][i]
        y = source.data["y"][i]
        info_div.text = f"<b>Seleccionado:</b> {x} = {y}"
    else:
        info_div.text = "Selecciona una barra (Tap) o usa Box Select. Hover para ver valores."

file_input.on_change("value", on_file_change)
select.on_change("value", on_select_change)
slider.on_change("value", on_slider_change)
source.selected.on_change("indices", on_selection_change)

curdoc().add_root(column(title_div, hint_div, error_div, file_input, select, slider, info_div, p))
curdoc().title = "CSV → Bokeh Interactivo"
"""
open("main.py","w").write(code)
print("✅ main.py escrito (fix: dict(new_source.data) y list(...)).")


✅ main.py escrito (fix: dict(new_source.data) y list(...)).


In [ ]:
# Inicia Bokeh en :5006 y publica una URL externa con Cloudflared
import os, subprocess, time, re, select, socket

def wait_port(port: int, timeout: int = 30) -> bool:
    end = time.time() + timeout
    while time.time() < end:
        s = socket.socket(); s.settimeout(1.0)
        try:
            s.connect(("127.0.0.1", port)); s.close(); return True
        except Exception:
            time.sleep(0.3)
    return False

# Cierra si había procesos previos
for name in ["bokeh_proc","cfd_proc"]:
    try: globals()[name].terminate()
    except: pass

# 1) Bokeh Server (logs on para diagnóstico)
bokeh_cmd = ["python","-m","bokeh","serve","main.py","--port","5006",
             "--allow-websocket-origin","*","--use-xheaders","--log-level","info"]
bokeh_proc = subprocess.Popen(bokeh_cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
assert wait_port(5006, 25), "Bokeh no levantó en :5006"

# 2) Túnel Cloudflared
cfd_cmd = ["./cloudflared","tunnel","--url","http://localhost:5006","--no-autoupdate"]
cfd_proc = subprocess.Popen(cfd_cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

url, buf = None, ""
pat = re.compile(r"https://[a-z0-9-]+\.trycloudflare\.com", re.I)
end = time.time() + 30
while time.time() < end and url is None:
    if cfd_proc.poll() is not None: break
    r,_,_ = select.select([cfd_proc.stdout], [], [], 0.5)
    if r:
        line = cfd_proc.stdout.readline()
        buf += line
        m = pat.search(line)
        if m: url = m.group(0)

if not url:
    try: cfd_proc.terminate()
    except: pass
    raise RuntimeError("No se obtuvo URL de cloudflared.\nLOG:\n" + buf)

print("🌐 URL base (index de Bokeh):", url + "/")
print("✅ URL de TU APP (abre esta):", url + "/main")

# Mostrar unas líneas de log de Bokeh por si hay errores de carga
print("\n— Logs iniciales de Bokeh —")
for _ in range(30):
    try:
        line = bokeh_proc.stdout.readline()
        if not line: break
        print(line.rstrip())
    except Exception:
        break


🌐 URL base (index de Bokeh): https://representative-leaving-curves-taxi.trycloudflare.com/
✅ URL de TU APP (abre esta): https://representative-leaving-curves-taxi.trycloudflare.com/main

— Logs iniciales de Bokeh —
<frozen runpy>:88: UserWarning:
It looks like you might be running the main.py of a directory app directly.
If this is the case, to enable the features of directory style apps, you must
call "bokeh serve" on the directory instead. For example:

    bokeh serve my_app_dir/

If this is not the case, renaming main.py will suppress this warning.

2025-09-15 13:36:08,613 Starting Bokeh server version 3.5.0 (running on Tornado 6.4.2)
2025-09-15 13:36:08,614 Host wildcard '*' will allow connections originating from multiple (or possibly all) hostnames or IPs. Use non-wildcard values to restrict access explicitly
2025-09-15 13:36:08,615 User authentication hooks NOT provided (default user enabled)
2025-09-15 13:36:08,619 Bokeh app running at: http://localhost:5006/main
2025-09-15 13

In [ ]:
# Celda 3 — Detener todo (ultra-robusta con timeouts; no se queda colgada)
import os, signal, time, subprocess, sys

def safe_terminate(proc, label="proc"):
    try:
        proc.terminate()
    except Exception:
        pass
    # Cerrar pipes para evitar bloqueos
    try:
        if getattr(proc, "stdout", None):
            proc.stdout.close()
    except Exception:
        pass
    try:
        if getattr(proc, "stderr", None):
            proc.stderr.close()
    except Exception:
        pass
    # Espera corta y luego kill
    try:
        proc.wait(timeout=1.0)
    except Exception:
        try:
            proc.kill()
        except Exception:
            pass

# 1) Intentar cerrar Popen guardados en variables (si existen)
for var in ("bokeh_proc", "cfd_proc"):
    proc = globals().get(var)
    if proc:
        safe_terminate(proc, var)
        try:
            del globals()[var]
        except Exception:
            pass

def run_quick(cmd):
    """Ejecuta un comando con timeout corto, sin bloquear la celda."""
    try:
        subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT,
                       timeout=2, check=False)
    except subprocess.TimeoutExpired:
        pass

# 2) Matar por nombre (rápido y con timeout)
# (Las comillas/expresiones evitan que 'grep' se mate a sí mismo)
for pat in [
    "cloudflared",
    "python -m bokeh",
    "bokeh serve",
    "main.py",          # por si quedó un python main.py
]:
    run_quick(["bash","-lc", f"pkill -9 -f '{pat}' || true"])

# 3) Matar por puerto (5006 = Bokeh). Usamos 'ss' si está disponible.
run_quick(["bash","-lc",
    r"(command -v ss >/dev/null 2>&1 && "
    r"ss -lptn 'sport = :5006' | awk -F, '/pid=/{gsub(/pid=/,\"\"); print $2}' "
    r"| awk '{print $1}' | xargs -r -n1 kill -9) || true"])

print("✅ Servidores y túnel detenidos (o no había nada que detener).")
